# Lab 2 Exercises for COMP 691 (Deep Learning)

In this lab we will learn some basics of Pytorch. 
- You will implement a feedforward neural network using different implementation styles.
- Understand how to use torch autograd for calculating gradients.
- Learn how to use GPUs for computation speed.

Save your answers for this lab as they will be used for part of Lab 3. 

Start by making a **copy** of this notebook in your Google Colab.


# Exercise 1: Loading the dataset

Below we will create a dataloader for the MNIST training data using torchvision package (following e.g. https://github.com/pytorch/examples/blob/master/mnist/main.py#L112-L120). 

The dataloader iterates over the training set and will output **mini-batches of size 256** image samples. 

**Note**: you do not need to use the image labels in the rest of this lab since you will not be doing any training. 

Remarks about using GPU:

- The "device" variable allows us to select which device to place the data on. Modify your colab (or local environment) to use a GPU.

- To use GPU in Google Colab, go to Runtime then choose "change runtime type". Then choose the hardware accelerator as GPU. 

- In your Google Colab notebook set the variable device to "cuda", rerunning the cell below such that the data is placed on GPU inside the for loop.


In [2]:
import torch
from torchvision import datasets,transforms
print(torch.__version__)
print(torch.cuda.is_available())
dataset1 = datasets.MNIST('../data', train=True, download=True, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset1, 
                                           batch_size=256, 
                                           shuffle=True,
                                           drop_last=True)

device='cuda'

for (data, target) in train_loader:
  data = data.to(device)
  target = target.to(device)
print(data.shape)
print(target.shape)

1.13.1
True
torch.Size([256, 1, 28, 28])
torch.Size([256])


If you ran the code cell above, you will notice that the data is a tensor of shape ([256, 1, 28, 28]) = (batch_size, number of color channels, length of image in pixels, width of image in pixels)

# Exercise 2: Building a neural network from the ground up!

Network Architecture: 
- Using only torch primitives (e.g. [torch.matmul](https://pytorch.org/docs/stable/generated/torch.matmul.html), [torch._relu](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html), etc) implement a simple feedforward neural network with 2 hidden layers that takes as input MNIST digits (28x28) and outputs **a single scalar value** i.e., the class. Avoid using any functions from torch.nn class. 


- You may select the hidden layer width (greater than 20) and activations (tanh, relu, sigmoid, others) as desired.  

- A typical layer will transform its inputs as follows: $y = σ (Wx+b) $, where $σ$ is the non-linear activation function.

- Initialize the weights  with [uniform random values](https://pytorch.org/docs/stable/generated/torch.rand.html) in the range -1 to 1 and [biases at 0](https://pytorch.org/docs/stable/generated/torch.zeros.html). 

Data:

Using the data obtained from Exercise 1, make a forward pass through the dataset in mini-batches of 256 (feed the network data). To check you are on the right track, the shape of your output should be ([256]). 

**Hint:** Remember that the goal is to feed the MNIST images and get a class label for each image. In this exercise there is no training so do not expect that the label will be meaningful/correct!

Pay attention to the shape of the input and how it gets changed as it passes from one layer to the next in the forward pass. Ex: (256, 28*28) -> (256, hiddden_size_1) -> (256,hidden_size_2) -> (256,1). This will help you when constructing the layers of the network.



In [3]:
import torch
## Initialize and track the parameters using a list or dictionary (modify the None)
h1 = 50
h2 = 50
param_dict = {
    "W0": torch.rand(784, h1) * 2 -1,
    "b0": torch.zeros(h1),
    "W1": torch.rand(h1, h2) * 2 -1,
    "b1": torch.zeros(h2),
    "W2": torch.rand(h2, 10) * 2 -1,
    "b2": torch.zeros(10),
    }

## Make sure your parameters in param_dict require gradient for training the network later! 
for name, param in param_dict.items():
    param_dict[name] = param.to(device)
    param.requires_grad=True


## Define the network
def my_nn(input, param_dict):
    r"""Performs a single forward pass of a Neural Network with the given 
    parameters in param_dict.

    Args:
        input (torch.tensor): Batch of images of shape (B, H, W), where B is 
            the number of input samples,and H and W are the image height and 
            width respectively.
        param_dict (dict of torch.tensor): Dictionary containing the parameters
            of the neural network. Expects dictionary keys to be of format 
            "W#" and "b#" where # is the layer number.

    Returns:
        torch.tensor: Neural network output of shape (B, )
    """
    # Reshape the input image from HxW to a flat vector of size H*W
    x = input.view(-1 , 28*28) #256*28*28 
    #Your code here
    
    relu = torch.nn.ReLU()
    x = torch.matmul(x, param_dict['W0']) + param_dict['b0']
    x = relu(x)
    
    x = torch.matmul(x, param_dict['W1']) + param_dict['b1']
    x = relu(x)
    
    x = torch.matmul(x, param_dict['W2']) + param_dict['b2']
    
    return relu(x)

cuda = torch.cuda.is_available()
## Perform forward pass
for (data, target) in train_loader:
    if cuda:
        data = data.to(device)
        target = target.to(device)
    #forward pass 
    output = my_nn(data, param_dict)
    print(output)

tensor([[ 38.4083,  30.1713, 101.5661,  ...,   0.0000,   7.1201,   0.0000],
        [  0.0000,  42.7322, 156.7559,  ...,   0.0000,  14.7621,   0.0000],
        [ 30.8171,   0.0000, 110.3756,  ...,   0.0000,   0.0000,   9.5616],
        ...,
        [  0.0000,  41.7566,  40.1210,  ...,  12.7773,   0.0000,   0.0000],
        [ 27.4027,  16.3040,  68.3748,  ...,   0.0000,   0.0000,   0.0000],
        [ 29.8133,  38.6984,  31.8596,  ...,   0.0000,   7.8673,   0.0000]],
       device='cuda:0')
tensor([[ 13.8763,  18.7843,  56.7430,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,  31.0008,  29.4131,  ...,  13.2964,   0.0000,   9.8300],
        [ 26.8281,  58.5034,  66.7701,  ...,   0.0000,   0.0000,   0.0000],
        ...,
        [ 50.2426,  59.1629, 123.6514,  ...,   0.0000,   5.9422,   0.0000],
        [  0.0000,  35.2053,  74.3565,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,   1.3737,  68.2902,  ...,   5.4350,  15.4415,  13.7493]],
       device='cuda:0')
tensor([[4.1

tensor([[4.5905e+01, 6.8870e+01, 8.0647e+01,  ..., 0.0000e+00, 0.0000e+00,
         1.7135e+00],
        [0.0000e+00, 7.4259e+01, 4.0769e+01,  ..., 0.0000e+00, 2.5188e+00,
         0.0000e+00],
        [1.0598e+01, 5.9986e+01, 1.0245e+02,  ..., 3.1151e-02, 2.9475e+01,
         0.0000e+00],
        ...,
        [8.5985e+00, 6.6116e+01, 5.6834e+01,  ..., 0.0000e+00, 2.2966e+01,
         0.0000e+00],
        [1.4763e+01, 3.2967e+01, 5.6258e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 2.8505e+01, 1.3584e+02,  ..., 0.0000e+00, 1.6574e+01,
         0.0000e+00]], device='cuda:0')
tensor([[1.1429e+01, 1.5719e+01, 4.4635e+01,  ..., 2.9010e+00, 0.0000e+00,
         2.2385e-01],
        [1.5236e+01, 5.4232e+01, 1.3885e+02,  ..., 0.0000e+00, 0.0000e+00,
         1.7405e-02],
        [0.0000e+00, 6.6997e+01, 7.1097e+01,  ..., 0.0000e+00, 5.5794e+00,
         0.0000e+00],
        ...,
        [3.3441e+00, 8.3646e+00, 2.0660e+01,  ..., 0.0000e+00, 8.9290e+00,
        

tensor([[  6.8631,  44.3827, 194.5475,  ...,   0.0000,  38.8847,   0.0000],
        [  0.0000,  27.8995,  88.7591,  ...,   5.3446,   0.0000,   0.0000],
        [  4.3146,   4.2892,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        ...,
        [ 48.4365, 112.9488, 167.0067,  ...,   0.0000,   3.4819,   0.0000],
        [  0.0000,  21.2368,  29.3790,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,  45.9560,  76.6582,  ...,   2.7250,   0.0000,   0.0000]],
       device='cuda:0')
tensor([[  0.0000,  26.7186,  96.8500,  ...,   0.0000,  29.7497,   0.0000],
        [  0.0000,  95.3580, 112.7062,  ...,   0.0000,  12.7858,   0.0000],
        [  8.2218,   5.7558,  79.6029,  ...,   0.0000,   0.0000,   0.8756],
        ...,
        [  2.7448,  17.4650,  32.8225,  ...,   0.0000,   1.9304,  11.6850],
        [  0.0000,   0.0000,  45.7903,  ...,   2.7246,   0.0000,   0.0000],
        [ 24.9402,  18.5156,  65.8235,  ...,   6.4349,  34.5241,   3.7078]],
       device='cuda:0')
tensor([[ 39

tensor([[  0.0000,  67.5995, 122.9159,  ...,   0.0000,   0.0000,   0.0000],
        [ 15.9857,  52.3670, 115.3040,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,  63.2480,  74.5780,  ...,   0.0000,   0.0000,   0.0000],
        ...,
        [  0.0000,  35.1735,  93.3540,  ...,   0.0000,  15.9155,   0.0000],
        [  0.0000,   9.7600,  92.1278,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,  32.1259, 136.1313,  ...,   0.0000,   0.0000,   0.0000]],
       device='cuda:0')
tensor([[ 28.1853,  33.0916,  63.7903,  ...,   0.0000,   2.9471,   0.0000],
        [  0.0000,  43.4145, 105.1813,  ...,   0.0000,   0.0000,   0.0000],
        [  4.5147,  74.4283,  26.3406,  ...,   9.6519,  10.3435,   0.0000],
        ...,
        [ 30.4004,  12.4124,  34.5014,  ...,   0.0000,   0.0000,   0.0000],
        [ 47.9261,  91.3659, 161.1597,  ...,   0.0000,  56.8220,   0.0000],
        [  0.0000,  18.0040,  26.9435,  ...,  26.8373,   0.0000,   0.0000]],
       device='cuda:0')
tensor([[ 21

tensor([[  1.3216,  93.3475,  82.9552,  ...,   0.0000,   0.0000,   0.0000],
        [  5.6774,  59.8263,  83.5644,  ...,   0.0000,  11.7548,   0.0000],
        [  0.0000,  22.2141,  54.6262,  ...,   2.9869,   7.1747,   0.0000],
        ...,
        [ 26.5386,  77.1751,  90.0557,  ...,   0.0000,  11.8377,   0.0000],
        [  0.0000,  70.0276, 171.9011,  ...,   0.0000,  25.8505,   0.0000],
        [  0.0000,  75.5509, 139.8886,  ...,   0.0000,   0.0000,   0.0000]],
       device='cuda:0')
tensor([[ 27.7319,  24.7199,  32.7422,  ...,   0.0000,   0.0000,   0.0000],
        [ 56.4899,  73.5919, 128.7668,  ...,   0.0000,  13.0083,   0.0000],
        [ 24.6085,  25.7505,  45.7770,  ...,   0.0000,   0.0000,   0.0000],
        ...,
        [  6.0111,  11.5735,  71.7801,  ...,   0.0000,  20.9987,   0.0000],
        [  0.0000,  94.7344,  90.0253,  ...,   0.0000,   0.0000,   0.0000],
        [ 57.4270,  25.1239, 107.2018,  ...,   0.0000,   0.0000,   0.0000]],
       device='cuda:0')
tensor([[  0

tensor([[ 13.1112,   0.0000, 119.4346,  ...,   0.0000,  21.2215,  17.2149],
        [  4.6369,   0.0000, 119.6304,  ...,  13.0255,  19.9530,   0.0000],
        [  6.7306,  72.1412,  36.7667,  ...,   0.0000,   0.0000,   0.0000],
        ...,
        [  3.1987,  61.1341, 141.8162,  ...,   0.0000,  23.3935,   0.0000],
        [  0.0000,  51.4234,  22.9496,  ...,   0.0000,   0.0000,   0.0000],
        [ 35.2018,   2.9803,  29.5657,  ...,  21.2871,   0.0000,   0.0000]],
       device='cuda:0')
tensor([[  0.0000,  73.0619, 209.1532,  ...,   0.0000,   0.0000,   0.0000],
        [  9.1883, 102.6611, 115.7788,  ...,   8.7061,  15.6297,   0.0000],
        [  0.0000,   0.0000,  89.7402,  ...,   0.0000,   0.0000,   0.0000],
        ...,
        [  0.0000,  83.1898, 146.2648,  ...,   0.0000,  11.5106,   0.0000],
        [  0.0000,  91.5723, 137.7337,  ...,   0.0000,   0.0000,   0.0000],
        [ 15.4569,  40.8862, 150.1794,  ...,   6.3823,  75.3648,   0.0000]],
       device='cuda:0')
tensor([[ 34

tensor([[  0.0000,   0.0000,  12.2359,  ...,  15.4054,   0.0000,  13.8962],
        [  0.0000,   7.5178,  27.5722,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,  44.6623,  55.5422,  ...,   0.9925,   0.2630,   0.0000],
        ...,
        [ 26.9176,  22.7224,  79.5972,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,  90.8656, 131.8966,  ...,   0.0000,   0.0000,   0.0000],
        [ 26.6657,  48.0230,  39.6170,  ...,   6.3603,   0.0000,   0.0000]],
       device='cuda:0')
tensor([[4.2622e+01, 8.7526e+01, 1.1038e+02,  ..., 0.0000e+00, 9.1660e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 9.5539e+01,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 9.0804e+01,  ..., 0.0000e+00, 1.0226e+01,
         0.0000e+00],
        ...,
        [5.6289e+01, 2.4645e+01, 1.0762e+02,  ..., 0.0000e+00, 1.7324e+01,
         0.0000e+00],
        [2.0849e+01, 1.5300e+01, 5.6201e+01,  ..., 4.0054e-04, 0.0000e+00,
         0.0000e+00],
        

# Exercise 3: Implementing the same network using torch.nn.module

Implement a new torch.nn.module that performs the equivalent of the network in Exercise 2 and call it "model". 

Initialize it with the same weights (ex: **nn.Linear**(in_features,out_features) so that you could have a fair comparison between the two networks. The way to do this is through **weight.data** = insert your desired weights. You can do a similar thing with the bias). 

Validate the outputs of this network is the same as the one in Exercise 2 on MNIST training set.

In [65]:
import torch.nn as nn
class myNet(nn.Module):
    def __init__(self):
        super(myNet, self).__init__()
        self.relu = torch.nn.ReLU()
        self.fc1 = nn.Linear(784, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = x.view(-1 , 28*28)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        return x

## Exercise 3.1: Validating that the two implementations are equal.

First you will need to make sure the param_dict from Exercise 2 and the nn module version have the same parameters (weights and biases). 

You can do this for example using: "**model.linear1.weight.data** = copy.deepcopy(param_dict['W0'].data.T)". 

**Note**: that we do a deepcopy just to make sure this model is separate from the one in the above cell

In [66]:
#Copy parameters from param_dict to nn module parameters
import copy
model = myNet()

model.fc1.weight.data = copy.deepcopy(param_dict['W0'].data.T)
model.fc1.bias.data = copy.deepcopy(param_dict['b0'].data.T)
model.fc2.weight.data = copy.deepcopy(param_dict['W1'].data.T)
model.fc2.bias.data = copy.deepcopy(param_dict['b1'].data.T)
model.fc3.weight.data = copy.deepcopy(param_dict['W2'].data.T)
model.fc3.bias.data = copy.deepcopy(param_dict['b2'].data.T)

#Run the assert statement below to check they match
for i,(data, _) in enumerate(train_loader):
    data = data.to(device)
    assert(((model(data)-my_nn(data, param_dict))**2).mean()<1e-4) # check that all the outputs are roughly equal
print("All Clear !")

All Clear !


## Exercise 4: Calculating gradients. 

For a single mini-batch of 256 samples (you can select any minibatch), compute the gradient of the average of the neural network outputs (over the minibatch) w.r.t to the weights. 

### Let's break this down:

First you will need to get the mean/average of the outputs. Then you need find the gradient of this mean w.r.t to the weights.

To find the gradient you can use torch autograd, which you can use simply it by calling **.backward()** on the desired variable.

Your task is to print the gradients for the first layer weight and bias. You can use either the model defined from exercise 2 or 3 for this.  

**Note**: The network here is $f: \mathcal{R}^{HW}\rightarrow\mathcal{R}$, which means that your input layer has $HW$ neurons ($HW$ features) and your output layer has one output neuron (one scalar output = class). Since each batch has $256$ samples, the mean can be obtained by $o=\frac{1}{256}\sum_{i=0}^{255}f(x_i)$ or simply calling **.mean()** on the output of the network. You are asked to find $\nabla_w o$ and $\nabla_b o$. To access the gradient of each parameter you can call **.grad**.

In [80]:

for (data, target) in train_loader:
    data = data.to(device)
    output = model(data)
    o_mean = output.mean()#get the mean/average of the outputs
    o_mean.backward()#find the gradient of this mean w.r.t to the weights.
    print("f1",model.fc1.weight.grad)
    print("b1",model.fc1.bias.grad)





f1 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
b1 tensor([ 1.5047e+02, -9.4648e+01,  8.1002e+01, -1.4784e+01,  6.7889e+02,
        -2.1688e+02, -6.7633e+00,  4.7815e+01,  2.4645e+02,  1.8356e+02,
        -1.3491e+02, -9.6987e+00, -3.1494e+01,  1.4889e+02,  5.5651e+02,
        -3.7453e+01,  2.8273e+02, -3.3036e+02, -2.3383e+02,  3.6322e+01,
         2.4145e+02,  5.5737e+02,  2.2789e+02, -9.1793e+01,  1.2090e+02,
        -3.2682e+02, -7.8497e+01,  2.0698e+02, -1.8312e+02,  1.6362e+01,
         5.3968e+02,  2.7834e+02, -8.1358e+01,  5.8730e-02,  1.4492e+02,
         3.5897e+01, -2.6670e+02, -1.6630e+01, -6.6301e-01,  4.5828e+01,
         1.9338e+02, -1.4666e+02,  2.6006e+02,  3.5740e+02,  1.3054e+02,
         4.7611e+01,  7.2877e+02,  3.9632e+02,  1.0990e+01,  1.040

b1 tensor([ 1.5385e+02, -9.6832e+01,  8.2834e+01, -1.5130e+01,  6.9420e+02,
        -2.2168e+02, -6.9008e+00,  4.8863e+01,  2.5204e+02,  1.8769e+02,
        -1.3788e+02, -9.8897e+00, -3.2245e+01,  1.5219e+02,  5.6910e+02,
        -3.8247e+01,  2.8916e+02, -3.3782e+02, -2.3904e+02,  3.7237e+01,
         2.4687e+02,  5.6989e+02,  2.3301e+02, -9.3850e+01,  1.2362e+02,
        -3.3432e+02, -8.0292e+01,  2.1161e+02, -1.8725e+02,  1.6733e+01,
         5.5171e+02,  2.8466e+02, -8.3168e+01,  5.1449e-02,  1.4817e+02,
         3.6674e+01, -2.7252e+02, -1.6972e+01, -6.8430e-01,  4.6881e+01,
         1.9783e+02, -1.4998e+02,  2.6588e+02,  3.6543e+02,  1.3361e+02,
         4.8668e+01,  7.4524e+02,  4.0510e+02,  1.1258e+01,  1.0648e+02],
       device='cuda:0')
f1 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ...,

f1 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
b1 tensor([ 1.5744e+02, -9.8969e+01,  8.4712e+01, -1.5493e+01,  7.1025e+02,
        -2.2681e+02, -7.0599e+00,  4.9954e+01,  2.5794e+02,  1.9189e+02,
        -1.4106e+02, -1.0122e+01, -3.3139e+01,  1.5561e+02,  5.8215e+02,
        -3.9159e+01,  2.9587e+02, -3.4548e+02, -2.4445e+02,  3.8035e+01,
         2.5252e+02,  5.8295e+02,  2.3857e+02, -9.5992e+01,  1.2644e+02,
        -3.4199e+02, -8.2127e+01,  2.1636e+02, -1.9150e+02,  1.7193e+01,
         5.6448e+02,  2.9116e+02, -8.5061e+01,  6.2652e-02,  1.5162e+02,
         3.7321e+01, -2.7861e+02, -1.7481e+01, -6.9165e-01,  4.7887e+01,
         2.0222e+02, -1.5347e+02,  2.7202e+02,  3.7369e+02,  1.3676e+02,
         4.9875e+01,  7.6236e+02,  4.1461e+02,  1.1388e+01,  1.089

f1 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
b1 tensor([ 1.6084e+02, -1.0111e+02,  8.6586e+01, -1.5734e+01,  7.2565e+02,
        -2.3184e+02, -7.1985e+00,  5.1081e+01,  2.6345e+02,  1.9601e+02,
        -1.4398e+02, -1.0343e+01, -3.3865e+01,  1.5898e+02,  5.9482e+02,
        -4.0027e+01,  3.0230e+02, -3.5295e+02, -2.4985e+02,  3.8821e+01,
         2.5798e+02,  5.9551e+02,  2.4380e+02, -9.8129e+01,  1.2920e+02,
        -3.4941e+02, -8.3968e+01,  2.2107e+02, -1.9568e+02,  1.7534e+01,
         5.7662e+02,  2.9743e+02, -8.6882e+01,  4.0134e-02,  1.5493e+02,
         3.8142e+01, -2.8461e+02, -1.7915e+01, -7.0345e-01,  4.8939e+01,
         2.0654e+02, -1.5679e+02,  2.7798e+02,  3.8193e+02,  1.3965e+02,
         5.0934e+01,  7.7901e+02,  4.2361e+02,  1.1586e+01,  1.112

f1 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
b1 tensor([ 1.6434e+02, -1.0332e+02,  8.8521e+01, -1.6078e+01,  7.4145e+02,
        -2.3687e+02, -7.2624e+00,  5.2283e+01,  2.6924e+02,  2.0026e+02,
        -1.4717e+02, -1.0606e+01, -3.4565e+01,  1.6245e+02,  6.0801e+02,
        -4.0919e+01,  3.0886e+02, -3.6074e+02, -2.5543e+02,  3.9698e+01,
         2.6351e+02,  6.0862e+02,  2.4903e+02, -1.0029e+02,  1.3199e+02,
        -3.5717e+02, -8.5803e+01,  2.2589e+02, -1.9988e+02,  1.7866e+01,
         5.8906e+02,  3.0395e+02, -8.8746e+01,  1.0291e-02,  1.5816e+02,
         3.9158e+01, -2.9102e+02, -1.8132e+01, -7.2224e-01,  5.0022e+01,
         2.1118e+02, -1.6022e+02,  2.8410e+02,  3.9035e+02,  1.4270e+02,
         5.1990e+01,  7.9604e+02,  4.3280e+02,  1.1896e+01,  1.137

f1 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
b1 tensor([ 1.6793e+02, -1.0548e+02,  9.0407e+01, -1.6544e+01,  7.5743e+02,
        -2.4185e+02, -7.5538e+00,  5.3426e+01,  2.7501e+02,  2.0462e+02,
        -1.5036e+02, -1.0828e+01, -3.5208e+01,  1.6591e+02,  6.2101e+02,
        -4.1874e+01,  3.1543e+02, -3.6853e+02, -2.6094e+02,  4.0495e+01,
         2.6925e+02,  6.2176e+02,  2.5433e+02, -1.0243e+02,  1.3484e+02,
        -3.6465e+02, -8.7729e+01,  2.3081e+02, -2.0421e+02,  1.8258e+01,
         6.0177e+02,  3.1052e+02, -9.0663e+01,  9.0667e-02,  1.6156e+02,
         3.9973e+01, -2.9731e+02, -1.8513e+01, -7.2951e-01,  5.1051e+01,
         2.1570e+02, -1.6363e+02,  2.9013e+02,  3.9885e+02,  1.4560e+02,
         5.3121e+01,  8.1309e+02,  4.4204e+02,  1.2156e+01,  1.162

f1 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
b1 tensor([ 1.7142e+02, -1.0768e+02,  9.2192e+01, -1.6881e+01,  7.7275e+02,
        -2.4702e+02, -7.7152e+00,  5.4495e+01,  2.8052e+02,  2.0883e+02,
        -1.5344e+02, -1.1045e+01, -3.5887e+01,  1.6919e+02,  6.3347e+02,
        -4.2640e+01,  3.2178e+02, -3.7607e+02, -2.6625e+02,  4.1363e+01,
         2.7482e+02,  6.3438e+02,  2.5944e+02, -1.0443e+02,  1.3756e+02,
        -3.7200e+02, -8.9445e+01,  2.3556e+02, -2.0847e+02,  1.8643e+01,
         6.1428e+02,  3.1685e+02, -9.2470e+01,  5.3188e-02,  1.6484e+02,
         4.0764e+01, -3.0339e+02, -1.8931e+01, -7.4011e-01,  5.2173e+01,
         2.2009e+02, -1.6693e+02,  2.9601e+02,  4.0680e+02,  1.4859e+02,
         5.4173e+01,  8.2965e+02,  4.5127e+02,  1.2411e+01,  1.184

b1 tensor([ 1.7470e+02, -1.0982e+02,  9.4018e+01, -1.7206e+01,  7.8815e+02,
        -2.5181e+02, -7.8602e+00,  5.5543e+01,  2.8606e+02,  2.1303e+02,
        -1.5658e+02, -1.1266e+01, -3.6556e+01,  1.7281e+02,  6.4602e+02,
        -4.3469e+01,  3.2825e+02, -3.8344e+02, -2.7151e+02,  4.2178e+01,
         2.8029e+02,  6.4698e+02,  2.6455e+02, -1.0656e+02,  1.4034e+02,
        -3.7933e+02, -9.1145e+01,  2.4026e+02, -2.1259e+02,  1.9002e+01,
         6.2654e+02,  3.2309e+02, -9.4419e+01,  8.1402e-02,  1.6818e+02,
         4.1676e+01, -3.0952e+02, -1.9313e+01, -7.6807e-01,  5.3204e+01,
         2.2448e+02, -1.7027e+02,  3.0197e+02,  4.1486e+02,  1.5154e+02,
         5.5267e+01,  8.4603e+02,  4.6012e+02,  1.2757e+01,  1.2080e+02],
       device='cuda:0')
f1 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ...,

## Exercise 5: CPU or GPU ?

Below you will find code for comparing the speed of a model on CPU and GPU as well as comparing the speed of a forward pass to a forward/backward pass. Instantiate a version of your model from exercise 3 (preferably a larger version e.g. width 100 or 500) and run the timing code. 

Write 1-2 sentences to summarize your observations about the relative speed's of CPU/GPU and forward/backward

In [81]:
#Instantiate a model defined from (3) here
model = myNet()

In [82]:
#Run on CPU
import time as timer
data = data.to('cpu')
model.cpu()

print('Running on CPU')

start = timer.time()
for _ in range(10):
  model(data)
print("Time taken forward", timer.time() - start) 

start = timer.time()
for _ in range(10):
  out = model(data).mean()
  out.backward()
print("Time taken forward/backward", timer.time() - start) 

Running on CPU
Time taken forward 0.0013861656188964844
Time taken forward/backward 0.004135847091674805


In [83]:
#Run on GPU
#initialize cuda
data = data.to('cuda')
model.cuda()
model(data)
print('Running on GPU')


start = timer.time()
for _ in range(10):
  model(data)
torch.cuda.synchronize()
print("Time taken", timer.time() - start) 

start = timer.time()
for _ in range(10):
  out = model(data).mean()
  out.backward()
torch.cuda.synchronize()
print("Time taken forward/backward", timer.time() - start) 

Running on GPU
Time taken 0.0007913112640380859
Time taken forward/backward 0.0024785995483398438


Summary of observations here: The time taken of GPU is clearly shorter than CPU. In both forward and backward pass. 